# Convolutional Neural Networks trainieren & verbessern

## Convolutional Neural Networks für Bildklassifikation

### Aufgabe 1

Zu Beginn wollen wir das Klassifizieren der Ziffern des MNIST Datensatzes aufgreifen, was wir bereits mit Fully Connected Networks (FCNs) durchgeführt haben.

Bereite den Datensatz, die `DataLoader` und die Loss-Funktion vor.

In [ ]:
import torch
import numpy as np
import torchvision

mnist_train = torchvision.datasets.MNIST('data/', train=True, 
                                         transform=torchvision.transforms.ToTensor(),
                                         download=True)
mnist_test = torchvision.datasets.MNIST('data/', train=False, 
                                         transform=torchvision.transforms.ToTensor(),
                                         download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=16)

loss_fn = torch.nn.CrossEntropyLoss()

### Aufgabe 2

Die wichtigsten Bestandteile eines Convolutional Neural Networks (CNNs) sind die [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) Layer. Die Faltungskernel haben üblicherweise die Größe $3\times 3$. Im Verlaufe des Netzwerk wird das Bild immer weiter downgesampelt, indem beispielsweise bei der Faltung das Stride auf $2$ gesetzt wird oder Pooling Layer wie [`torch.nn.MaxPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) verwendet werden. Nach jedem Downsampling wird die Anzahl der Channels um einen Faktor von $2$ erhöht, um die Zahl der Neuronen nicht zu stark zu reduzieren.

Das letzte Layer ist fully connected, sodass man die Wahrscheinlichkeiten für die gewünschte Anzahl an Klassen erhält. Da ein solches Layer allerdings ein Batch von Vektoren erwartet, muss man die Feature vorher in Vektoren umwandeln. Eine Möglichkeit dies zu tun haben wir uns in der Übung zu FCNs angeschaut, eine andere ist [`torch.nn.AvgPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html).

Baue ein Netzwerk, was aus $5$-$10$ Layern besteht und zwei Downsampling Operationen enthält. Überprüfe, ob die Dimensionen der Layer zusammen passen, indem du ein Batch mit deinem Netzwerk verarbeitest.

In [ ]:
net = torch.nn.Sequential(torch.nn.Conv2d(1, 8, 3, padding=1), 
                          torch.nn.ReLU(), 
                          torch.nn.Conv2d(8, 8, 3, padding=1),
                          torch.nn.ReLU(), 
                          torch.nn.AvgPool2d(2, 2), 
                          torch.nn.Conv2d(8, 16, 3, padding=1),
                          torch.nn.ReLU(),
                          torch.nn.Conv2d(16, 16, 3, padding=1),
                          torch.nn.ReLU(), 
                          torch.nn.AvgPool2d(2, 2),
                          torch.nn.Conv2d(16, 32, 3, padding=1),
                          torch.nn.ReLU(),
                          torch.nn.Conv2d(32, 32, 3, padding=1),
                          torch.nn.ReLU(),
                          torch.nn.Flatten(), 
                          torch.nn.Linear(7 * 7 * 32, 10))

x = mnist_train[0][0].reshape(1, 1, 28, 28)
out = net(x)
print(out)

### Aufgabe 3

Trainiere dein Netzwerk und werte die Accuracy aus dem Testset aus.

In [ ]:


def evaluate(model, loader):
    sum_correct = 0
    sum_imgs = 0

    for x_batch, y_batch in loader:
        # Vorhersage des Netzes ohne Gradientenberechnung
        with torch.no_grad():
            y_pred = model(x_batch)

        # Vorhergesagtes Label
        y_pred = torch.argmax(y_pred, 1)

        # Anzahl der Bilder updaten
        sum_imgs += len(x_batch)

        # Anzahl der korrekt klassifizierten Bilder
        sum_correct += torch.sum(y_pred == y_batch).item()

    # Accuracy berechnen und ausgeben
    accuracy = sum_correct / sum_imgs
    
    return accuracy

In [ ]:
from matplotlib import pyplot as plt
from tqdm import tqdm

# Erstellen des Optimizers
optimizer = torch.optim.Adam(net.parameters())

# Training
n_epoch = 6

loss_hist = []

for ep in range(n_epoch):
    for x_batch, y_batch in tqdm(train_loader):
        # Vorhersage des Netzwerks
        y_pred = net(x_batch)

        # Loss berechnen
        loss = loss_fn(y_pred, y_batch)

        # Backpropagation
        loss.backward()

        # Optimizer step
        optimizer.step()
        optimizer.zero_grad()

        # Loss speichern
        loss_hist.append(loss.item())
    
    # Accuracy bestimmen
    acc = evaluate(net, test_loader)
    print('Accuracy nach %i Epochen: %.2f%%' % (ep + 1, acc * 100))

plt.plot(loss_hist)
plt.show()

## Neuronale Netze evaluieren und verbessern

### Aufgabe 4

Die Confusion Matrix ist ein hilfreiches Tool, um die Fehler, die ein Netzwerk macht, zu verstehen. Sie kann man mit der Funktion `sklearn.metrics.confusion_matrix` ermitteln.

Bestimme die Confusion Matrix und stelle sie grafisch dar.

In [ ]:
import seaborn as sns
import pandas as pd

from sklearn.metrics import confusion_matrix

labels = []
predictions = []

with torch.no_grad():
    for x_batch, y_batch in tqdm(test_loader):
        y_pred = net(x_batch)
        y_pred_label = torch.argmax(y_pred, dim=-1)
        
        labels.extend(y_batch.numpy())
        predictions.extend(y_pred_label.numpy())
        

# Confusion Matrix bestimmen
conf_mat = confusion_matrix(labels, predictions)

# Grafische Darstellung
classes = list(range(10))
conf_mat_pd = pd.DataFrame(conf_mat, classes, classes)
plt.figure(figsize=(15, 9))
sns.heatmap(conf_mat_pd, annot=True)
plt.show()

In [ ]:
l = []
l.extend(np.array([1, 2, 3]))
l.extend(np.array([2, 4, 3]))

In [ ]:
l

### Aufgabe 5

Ziffern zu klassifizieren ist ein relativ einfaches Problem und man kann mit relativ einfachen Modellen bereits eine Genauigkeit von $99\%$ erreichen. Deswegen wollen wir jetzt mit dem komplexeren Datensatz CIFAR10 arbeiten, der Farbbilder der Größe $32\times 32$ Pixel aus $10$ Klassen wie Frosch und Schiff beinhaltet.

Lade den Datensatz mit [`torchvision.datasets.CIFAR10`](https://pytorch.org/vision/stable/generated/torchvision.datasets.CIFAR10.html) und visualisiere einige Bilder aus verschiedenen Klassen. Lass dir außerdem die Bezeichnungen der Klassen anzeigen.

In [ ]:
cifar10_train_ = torchvision.datasets.CIFAR10('data/', train=True, 
                                             transform=torchvision.transforms.ToTensor(),
                                             download=True)
cifar10_test = torchvision.datasets.CIFAR10('data/', train=False, 
                                            transform=torchvision.transforms.ToTensor(),
                                            download=True)

In [ ]:
cifar10_train, cifar10_val = torch.utils.data.random_split(cifar10_train_, 
                                                           [45000, 5000])

### Aufgabe 6

Teile den Trainingsdatensatz und ein Trainings- und einen Validierungsdatensatz auf, sodass zur Validierung etwa $10\%$ der ursprünglichen Trainingsdaten verwendet werden. Erstelle `DataLoader` für den Trainings-, Validierungs- und Testdatensatz.

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=16)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=16)

### Aufgabe 7

Konstruiere ein CNN für den CIFAR10-Datensatz. Da die Bilder größer sind als die von MNIST, solltest du $3$ Downsampling Operationen einbauen.

In [ ]:
net = # ???

x = cifar10_train[0][0].reshape(1, 3, 32, 32)
out = net(x)
print(out)

### Aufgabe 8

Trainiere dein CNN. Bestimme dabei die Genauigkeit nach jeder Epoche und stelle sie, den Loss, sowie die Confusion Matrix nach dem Training grafisch dar. 

Verwende bei der Evaluierung auschließlich den Validierungsdatensatz. Den Testdatensatz wollen wir erst ganz am Ende benutzen.

### Aufgabe 9

Versuche die Performance deines Modells zu verbessern, indem du beispielsweise die Architektur optimierst, Regularisierung oder Data Augmentation verwendest und Learning Rate Decay benutzt.

Beachte, dass Batch Normalization und Dropout Layer sich während des Trainings und der Evaluierung unterschiedlich verhalten haben, weswegen du deinem CNN explizit mit `net.train()` oder `net.eval()` mitteilen musst, in welcher Phase es sich befindet.

### Aufgabe 10

Bestimme nun die Genauigkeit und die Confusion Matrix auf dem Testset.